# Simulation of production line with defects - Dataset creation and Inference

_This notebook is originally created by [@paularamos](https://github.com/paularamo) for CVPR-2022 Tutorial [How to get quick and performant model for your edge application. From data to application](https://paularamo.github.io/cvpr-2022/)_

### Definitions

[Anomalib](https://github.com/openvinotoolkit/anomalib): Anomalib is a deep learning library that aims to collect state-of-the-art anomaly detection algorithms for benchmarking on both public and private datasets. Anomalib provides several ready-to-use implementations of anomaly detection algorithms described in the recent literature, as well as a set of tools that facilitate the development and implementation of custom models. The library has a strong focus on image-based anomaly detection, where the goal of the algorithm is to identify anomalous images, or anomalous pixel regions within images in a dataset.

[Dobot](https://en.dobot.cn/products/education/magician.html) The Magician is an education robot arm portable and capable to run various automation tasks. With an interface in C++ and python we can control the robot using this notebook. 

> NOTE: 
If you don't have the robot you can replace it by your custome problem. 

### Use case

Using the [Dobot Magician](https://www.dobot.cc/dobot-magician/product-overview.html) we could simulate a production line system. Imagine we have a cubes factory and they need to know when a defect piece appear in the process. We know very well what is the aspecto of the normal cubes. Defects are coming no often and we need to put those defect cubes out of the production line.

<img src="https://user-images.githubusercontent.com/10940214/174126337-b344bbdc-6343-4d85-93e8-0cb1bf39a4e3.png" alt="drawing" style="width:400px;"/>


| Class | Yellow cube | Red cube | Green cube | Inferencing using Anomalib
| --------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------ | --------------------------------------------------------------------------- | --------------------------------------------------------------------------- |
| Normal | <img src="https://user-images.githubusercontent.com/10940214/174083561-38eec918-efc2-4ceb-99b1-bbb4c91396b2.jpg" alt="drawing" style="width:150px;"/> | <img src="https://user-images.githubusercontent.com/10940214/174083638-85ff889c-6222-4428-9c7d-9ad62bd15afe.jpg" alt="drawing" style="width:150px;"/> | <img src="https://user-images.githubusercontent.com/10940214/174083707-364177d4-373b-4891-96ce-3e5ea923e440.jpg" alt="drawing" style="width:150px;"/> | <img src="https://user-images.githubusercontent.com/10940214/174129305-03d9b71c-dfd9-492f-b42e-01c5c24171cc.jpg" alt="drawing" style="width:150px;"/> |
| Abnormal | <img src="https://user-images.githubusercontent.com/10940214/174083805-df0a0b03-58c7-4ba8-af50-fd94d3a13e58.jpg" alt="drawing" style="width:150px;"/> | <img src="https://user-images.githubusercontent.com/10940214/174083873-22699523-22b4-4a55-a3da-6520095af8af.jpg" alt="drawing" style="width:150px;"/> | <img src="https://user-images.githubusercontent.com/10940214/174083944-38d5a6f4-f647-455b-ba4e-69482dfa3562.jpg" alt="drawing" style="width:150px;"/> | <img src="https://user-images.githubusercontent.com/10940214/174129253-f7a567d0-84f7-4050-8065-f00ba8bb973d.jpg" alt="drawing" style="width:150px;"/> | 

Using Anomalib we are expecting to see this result.


### Import packages for Anomalib and Dobot

In [9]:
#Anomalib imports
from __future__ import annotations

from pathlib import Path
from typing import Any, Tuple, List

import numpy as np
from IPython.display import display
from PIL import Image
from pytorch_lightning import Trainer
from torchvision.transforms import ToPILImage

from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.models import get_model
from anomalib.pre_processing.transforms import Denormalize
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks

# Dobot/general imports
import threading
import DobotDllType as dType # comment this line if you don't have the Dobot Magician
import cv2

import collections
import os
import sys
import time

import numpy as np
from IPython import display

from datetime import datetime

sys.path.append("../utils")
import notebook_utils as utils #for using video encoder in the jupyter notebook

import threading

### Helper funtions

Here you will find funtions to create filenames, capture images, run the inference and read the confidence of the detection.

In [ ]:
def filename_fc(acquisition: bool, folder: str, dataset_path: str): -> str
    """
    Create the filename for new data(images) or 
    the filename for inference result image

    :param: acquisition: mode True: acquire new data, 
                         False: run the inference
            folder: directory to save the new images in 
                    acquisition mode (abnormal or normal) 
            dataset_path: Initial path to save new images and results
    :returns:
            filename: captured image filename
            resultname: heatmap after inference filename
    """
    now = datetime.now()
    print(acquisition)
    print(folder)
    if not acquisition:
        print("In inference mode we cannot use this function")
        filename = None #Path(f"{dataset_path}/testing/inferencing_") + str(now.strftime("%Y%m%d%H%M%S")) + ".jpg"
        return filename 
    
    if folder == "abnormal":
        filename = Path(f"{dataset_path}/train/anormal/input_" + str(now.strftime("%Y%m%d%H%M%S")) + ".jpg"
    elif folder == "normal":
        filename = Path(f"{dataset_path}/train/normal/input_" + str(now.strftime("%Y%m%d%H%M%S")) + ".jpg"
    print(filename)
    return filename

### Prepare the mode (acquisition or inference mode) and define the work directory

In [ ]:
# Prepare the path to save the new image and the anomalib results
dataset_path = "C:/Anomalib/datasets/cubes"
# Acquisiton mode
acquisition = False #True False
# Normal or abnormal batch
folder = "normal" #abnormal normal

### Define some functions for using a webcam in real time

Using multi-threading we will open the video to auto-capture an image when the robot locates the cube in front of the camera.

In [ ]:
def getFrame():
    global frame, input_image
    while True:
        frame = player.next()
        if frame is None:
            print("Source ended")
            break
        # If the frame is larger than full HD, reduce size to improve the performance.
        scale = 1280 / max(frame.shape)
        if scale < 1:
            frame = cv2.resize(
                src=frame,
                dsize=None,
                fx=scale,
                fy=scale,
                interpolation=cv2.INTER_AREA,
            )
        # Get the results.
        input_image = np.array(frame)

def anomalib_inference(use_popup):
    global predictions
    while True:
        predictions = inferencer.predict(image=frame)
        if use_popup:
                cv2.imshow(winname=title2, mat=predictions.heat_map)
                key = cv2.waitKey(1)
                # esc = 27
                if key == 27:
                    break
            else:
                # Encode numpy array to jpg.
                _, encoded_img2 = cv2.imencode(
                    ext=".jpg", img=predictions.heat_map, params=[cv2.IMWRITE_JPEG_QUALITY, 100]
                )
                # Create an IPython image.
                i2 = display.Image(data=encoded_img2)
                # Display the image in this notebook.
                display.clear_output(wait=True)
                display.display(i2)
    
def create_dataset():
    while True:
        showPic = cv2.imwrite(filename,frame)
    
def realtime(use_popup):
    while True:
        if use_popup:
                cv2.imshow(winname=title, mat=frame)
                key = cv2.waitKey(1)
                # esc = 27
                if key == 27:
                    break
            else:
                # Encode numpy array to jpg.
                _, encoded_img = cv2.imencode(
                    ext=".jpg", img=frame, params=[cv2.IMWRITE_JPEG_QUALITY, 100]
                )
                # Create an IPython image.
                i = display.Image(data=encoded_img)
                # Display the image in this notebook.
                display.clear_output(wait=True)
                display.display(i)

### Using a webcam or a USB camera for running the inference

Connect and identify your US camera, we will use a a video player to embed the video in this notebook. 

> NOTE: 
If you don't have the robot you can replace it by your custom problem. See the comments below. 

In [ ]:
CON_STR = {
    dType.DobotConnect.DobotConnect_NoError:  "DobotConnect_NoError",
    dType.DobotConnect.DobotConnect_NotFound: "DobotConnect_NotFound",
    dType.DobotConnect.DobotConnect_Occupied: "DobotConnect_Occupied"}

#Load Dll and get the CDLL object
api = dType.load()

#Connect Dobot
state = dType.ConnectDobot(api, "", 115200)[0]
print("Connect status:",CON_STR[state])

use_popup = False #True

if (state == dType.DobotConnect.DobotConnect_NoError):

    Calibration__0__Run__1 = None
    Calibration_X = None
    Calibration_Y = None
    Calibration_Z = None
    Place_X = None
    Place_Y = None
    Place_Z = None
    Anomaly_X = None
    Anomaly_Y = None
    Anomaly_Z = None
    j = None
    k = None
    time_start = None
  
    print('[HOME] Restore to home position at first launch, please wait 30 seconds after turnning on the Dobot Magician.')
    print('[BLOCKS] Place them besides the non-motor side of the conveyor belt, the same side where the pick and place arm is.')
    print('[PLACING BLOCKS] Place the blocks by 3×3.')
    print('[CALIBRATION POINT] Looking from the back of Dobot, the top left block is the calibration point.')
    print('[CALIBRATION] Set the first variable to 0 to test the calibration point, then set 1 to start running.')
    print('[DIRECTION] Standing behind Dobot Magician facing its front direction, X is front and back direction, Y is left and right direction. ')
    print('[CONNECTION] Motor of the conveyor belt connects to port Stepper1.')
    
    Calibration__0__Run__1 = 1
    Calibration_X = 221.2288
    Calibration_Y = -117.0036
    Calibration_Z = -42.3512
    Place_X = 23.7489 #42.2995 #
    Place_Y = -264.2602 #-264.6927 #
    Place_Z = 18.0862 #63.65 #
    Anomaly_X = -112 #-84.287 #
    Anomaly_Y = -170 #-170.454 #
    Anomaly_Z = 90 #61.5359 #
    dType.SetEndEffectorParamsEx(api, 59.7, 0, 0, 1)
    j = 0
    k = 0
    dType.SetPTPJointParamsEx(api,400,400,400,400,400,400,400,400,1)
    dType.SetPTPCommonParamsEx(api,100,100,1)
    dType.SetPTPJumpParamsEx(api,40,100,1)
    dType.SetPTPCmdEx(api, 0, Calibration_X,  Calibration_Y,  Calibration_Z, 0, 1)
    dType.SetEndEffectorSuctionCupEx(api, 0, 1)
    STEP_PER_CRICLE = 360.0 / 1.8 * 10.0 * 16.0
    MM_PER_CRICLE = 3.1415926535898 * 36.0
    vel = float(0) * STEP_PER_CRICLE / MM_PER_CRICLE
    dType.SetEMotorEx(api, 1, 0, int(vel), 1)
    if Calibration__0__Run__1:
        player = None
        # Create a video player to play with target fps.
        player = utils.VideoPlayer(source=source, flip=flip, fps=30, skip_first_frames=skip_first_frames)
        # Start capturing.
        player.start()
        if use_popup:
            title = "Press ESC to Exit - Original"
            title2 = "Press ESC to Exit - Inference"
            cv2.namedWindow(
                winname=title, flags=cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE
            )
            cv2.namedWindow(
                winname=title2, flags=cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE
            )

        for count in range(9):
            dType.SetPTPCmdEx(api, 0, (Calibration_X - j),  (Calibration_Y - k),  (Calibration_Z - 10), 0, 1)
            dType.SetEndEffectorSuctionCupEx(api, 1, 1)
            #dType.dSleep(150)
            dType.SetPTPCmdEx(api, 0, (Place_X - 0),  (Place_Y - 0),  (Place_Z + 90), 0, 1)
            filename, resultname = filename_fc(acquisition, folder)
            ### Capture a frame from the video player - start thread
            gfthread = threading.Thread(target=getFrame, args='')
            gfthread.daemon = True
            gfthread.start()
            ### Display the video
            rtthread = threading.Thread(target=realtime(), args=(use_popup,))
            rtthread.daemon = True
            rtthread.start()
            if not(acquisition):
                # Get the inference results.
                aithread = threading.Thread(target=anomalib_inference, args=(use_popup,))
                aithread.daemon = True
                aithread.start()
                score = predictions.pred_score
                ###################################
                if score > 0.34:
                    dType.SetPTPCmdEx(api, 0, Anomaly_X,  Anomaly_Y,  Anomaly_Z, 0, 1) ### define point for abnormalities
                else:
                    dType.SetPTPCmdEx(api, 0, Place_X,  Place_Y,  Place_Z, 0, 1)

            if acquisition:
                filename = filename_fc(acquisition, folder, dataset_path)
                ### Create the dataset
                cdthread = threading.Thread(target=create_dataset(filename), args='')
                cdthread.daemon = True
                cdthread.start()
                dType.SetPTPCmdEx(api, 0, Place_X,  Place_Y,  Place_Z, 0, 1)
                
                #print("continue in the conveyor belt")

            dType.SetEndEffectorSuctionCupEx(api, 0, 1)
            #dType.dSleep(150)
            j = j + 25
            if j == 75:
                k = k + 25
                j = 0
            dType.SetPTPCmdEx(api, 7, 0,  0,  20, 0, 1)
            time_start = dType.gettime()[0]
            STEP_PER_CRICLE = 360.0 / 1.8 * 10.0 * 16.0
            MM_PER_CRICLE = 3.1415926535898 * 36.0
            vel = float(50) * STEP_PER_CRICLE / MM_PER_CRICLE
            dType.SetEMotorEx(api, 1, 1, int(vel), 1)
            filename = None
            score = 0
            while True:
                if (dType.gettime()[0]) - time_start >= 0.5 : # Time over conveyor belt
                    STEP_PER_CRICLE = 360.0 / 1.8 * 10.0 * 16.0
                    MM_PER_CRICLE = 3.1415926535898 * 36.0
                    vel = float(0) * STEP_PER_CRICLE / MM_PER_CRICLE
                    dType.SetEMotorEx(api, 1, 0, int(vel), 1)
                    break
        dType.SetEndEffectorSuctionCupEx(api, 0, 1)
        dType.SetPTPCmdEx(api, 0, Calibration_X,  Calibration_Y,  Calibration_Z, 0, 1)


In [ ]:
!python -VV